In [5]:
from gnutools import fs
from bpd.dask import DataFrame, udf
from bpd.dask import functions as F
from gnutools.remote import gdrivezip

In [6]:
# Import a sample dataset
gdrivezip("gdrive://1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE")
df = DataFrame({"filename": fs.listfiles("/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE", [".wav"])})
df.compute()      

,filename
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/919...
1,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/6a2...
2,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/682...
3,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/beb...
4,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/d37...
...,...
145,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/6a...
146,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e3...
147,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/68...
148,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e7...


In [7]:
# Register a user-defined function
@udf
def word(f):
    return fs.name(fs.parent(f))

# Display the dataframe
# Retrieve the first 3 filename per classe
@udf
def initial(classe):
    return classe[0]

@udf
def lists(classes):
    return list(set(classes))
    
from bpd.dask.pipelines import *

df.run_pipelines(
    [
        {
            select_cols: ("filename",),
            pipeline: (
                ("classe", word(F.col("filename"))),
                ("name", udf(fs.name)(F.col("filename"))),
            ),
        },
        {
            group_on: "classe",
            select_cols: ("name", ),
            pipeline: (
                ("initial", initial(F.col("classe"))),
            ),
        },
        {
            group_on: "initial",
            select_cols: ("classe", ),
            pipeline: (
                ("_initial", lists(F.col("classe"))),
            ),
        },
    ]
)\
.withColumnRenamed("_initial", "initial")\
.compute()

,filename,initial
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/919...,[wow]
1,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/6a2...,[wow]
2,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/682...,[wow]
3,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/beb...,[wow]
4,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/d37...,[wow]
...,...,...
13,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/6a...,[left]
14,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e3...,[left]
15,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/68...,[left]
16,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e7...,[left]
